---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="eb0dad00" -->
# Table des matières
- [Une approche par petits pas](#une-approche-par-petits-pas)
- [Recalage linéaire simple incluant la variable catégorielle](#recalage-linéaire-simple-incluant-la-variable-catégorielle)
- [Affichage des résultats](#affichage-des-résultats)
- [Interprétation des résultats](#interprétation-des-résultats)
- [Comparaison des profits réels et prédits](#comparaison-des-profits-réels-et-prédits)
- [Prédiction d'une valeur unique](#prédiction-dune-valeur-unique)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)



<!-- #region id="7a046863" -->
Dans cet exemple, nous allons reprendre le même contexte et jeu de données que dans l'exemple précédent, mais cette fois-ci, nous allons utiliser la variable catégorielle afin de montrer comment traiter ce type de variable.
<!-- #endregion -->

<!-- #region id="ec4dbb9e" -->
# <a id=une-approche-par-petits-pas>Une approche par petits pas</a>
<!-- #endregion -->

<!-- #region id="5d955cd7" -->
Dans ce qui suit, nous allons utiliser un modèle incluant toutes les variables du jeu de données des jeunes pousses. Nous allons voir comment intégrer des variables catégorielles à l'analyse. 

Notre modèle correspond donc à :
$$\text{Profit} \approx a_{0} + a_{1}\text{R&D} + a_{2}\text{Administration} + a_{3}\text{Marketing}+ a_{4}\text{Florida} + a_{5}\text{New York}$$

Puisque la variable `State` contiens trois catégories; elle peut être convertie en deux variables
nominales $\text{Florida}$ et $\text{New York}$. Les variables nominales étant mutuellement exclusives, nous n'avons pas besoin d'inclure une variable $\text{California}$, car elle est sélectionnée par élimination lorsque
$\text{Florida}=0$ et $\text{New York}=0$.
<!-- #endregion -->

<!-- #region id="b5934bf3" -->
# <a id=recalage-linéaire-simple-incluant-la-variable-catégorielle>Recalage linéaire simple incluant la variable catégorielle</a>
<!-- #endregion -->

<!-- #region id="99df0c50" -->
Commençons par importer les données, comme dans l'exemple précédent, en séparant les variables X et la réponse Y du jeu de données. Cette fois, nous allons prendre l'ensemble des colonnes.
<!-- #endregion -->



In [ ]:
# Lecture du jeu de données en format CSV.
dataset = pd.read_csv("/pax/shared/GIF-U014/50_Startups.csv")

# Affichage des cinq premières lignes du fichier.
dataset.head()


In [ ]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, 4]



<!-- #region id="ade5ba67" -->
Puisqu'on utilise l'information géographique contenue dans la variable `State`, il faut convertir la variable
$\text{State}={\text{New York}, \text{California}, \text{Florida}}$ en deux variables nominales $\text{Florida}$ et $\text{New York}$. 

Si l'on utilisait trois variables nominales au lieu de deux, on aurait un jeu de variables corrélées
entre elles puisque la valeur de la variable $\text{California}$ se déduit de celles de $\text{Florida}$ et $\text{New York}$.
Plusieurs méthodes de régressions sont sensibles au problème de covariance entre les données.
C'est une leçon importante à retenir.

On utilise la fonction
[`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) de la librairie Pandas pour générer les variables.

<!-- #endregion -->



In [ ]:
states = pd.get_dummies(X["State"], drop_first=True)

X = X.drop("State", axis=1)
X = pd.concat([X, states], axis=1)



<!-- #region id="384298d3" -->
Affichons les premières lignes des variables X afin de voir nos nouvelles variables.
<!-- #endregion -->



In [ ]:
print("\nVariables utilisées (cinq premières lignes des données) :\n")
print(X.head())



<!-- #region id="aadd8007" -->
Il y a maintenant cinq colonnes puisque les variables nominales sont utilisées. Il y a plusieurs
ordres de grandeur entre les différentes variables; raison de plus pour normaliser les données.

La section de code suivante traite de l'entraînement et du test du modèle de régression. Elle est identique à celle
utilisée pour le premier modèle. Elle est répétée ici sous forme plus condensée.
<!-- #endregion -->



In [ ]:
# Séparation des données en un ensemble d'entraînement (80 % des données) et
# de test (20 % des données)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Sélectionnons une fonction de normalisation et entrainons-la avec les
# données d'entraînement
norm = StandardScaler()
X_train = norm.fit_transform(X_train)
X_test = norm.transform(X_test)

# Entraînement du modèle de régression linéaire sans interaction
reg = LinearRegression()
reg.fit(X_train, y_train)

# Calculons la réponse prédite pour les données de test.
y_pred = reg.predict(X_test)

# Calcul de la métrique R2 à partir des vraies valeurs et les valeurs prédites
# pour les données de test.
score = r2_score(y_test, y_pred)



<!-- #region id="91d84352" -->
# <a id=affichage-des-résultats>Affichage des résultats</a>
<!-- #endregion -->



In [ ]:
print("Coefficients: \n")
print("\t a_{0} = %f\n" % (reg.intercept_))
for i, coeff in enumerate(reg.coef_):
    print("\t a_{%d} = %f\n" % (i + 1, reg.coef_[i]))
print("\nMétrique R2:  %0.1f %%\n" % (100 * score))



<!-- #region id="a2f9dc03" -->
# <a id=interprétation-des-résultats>Interprétation des résultats</a>
<!-- #endregion -->

<!-- #region id="2d25c36b" -->
Selon la métrique $R^2$, le second modèle explique $93,5~\%$ de l'information dans les données. C'est légèrement inférieur au modèle précédent, mais probablement dû à des fluctuations statistiques. Les résultats sont essentiellement comparables.

Les principales variables en ordre d'importance n'ont pas changé, soit `R&D Spend`, `Marketing Spend` et `Administration`.

On voit que les valeurs de $\alpha_{4}$ et $\alpha_{5}$ sont faibles par rapport aux autres. Ainsi, l'effet
géographique est moins important que les dépenses en administration!

Comment interpréter les signes de $\alpha_{4}$ ($\text{Florida}$) et $\alpha_{5}$ ($\text{New York}$)? 
La variable nominale qui n'apparaît pas, $\text{California}$, sert de référence. L'interprétation est la suivante:

- $\alpha_{4}<0$ signifie que les profits diminuent si l'on investit en $\text{Floride}$ plutôt qu'en $\text{Californie}$,
- $\alpha_{5}>0$ signifie que les profits augmentent si l'on investit à $\text{New York}$ plutôt qu'en $\text{Californie}$.

Attention encore une fois; il faut se rappeler que des corrélations entre les variables n'impliquent pas nécessairement des liens de causalité. Peut-être que les taxes sont différentes entre les états. Les différences de profit ne seraient pas expliquées par une meilleure performance des compagnies de New York, mais simplement dues à des taxes plus faibles dans cet état.
<!-- #endregion -->

<!-- #region id="266950f4" -->
# <a id=comparaison-des-profits-réels-et-prédits>Comparaison des profits réels et prédits</a>
<!-- #endregion -->



In [ ]:
# Conversion en k$
y_test = y_test / 1000
y_pred = y_pred / 1000

fig = plt.figure(figsize=(8, 6))
plt.plot(y_test, y_pred, "o")
plt.plot(y_test, y_test, "-r", label="Prédiction parfaite")
plt.title("Modèle utilisant l'information géographique", fontsize=16)
plt.xlabel("Profit réel (k$)", fontsize=16)
plt.ylabel("Profit prédit (k$)", fontsize=16)
plt.legend()
fig.tight_layout()
plt.show()



<!-- #region id="4f992627" -->
Les prédictions du second modèle sont aussi bonnes que pour le premier modèle, plus simple.
<!-- #endregion -->

<!-- #region id="d2ce007b" -->
# <a id=prédiction-dune-valeur-unique>Prédiction d'une valeur unique</a>
<!-- #endregion -->

<!-- #region id="68758757" -->
Jusqu'à maintenant, on a effectué des prédictions sur les ensembles d'entraînement et de test
qui contenaient les données de plusieurs compagnies. Voici comment le faire pour une compagnie à la fois.

Quel serait le profit d'une compagnie californienne ayant investi $50 \text{k}~\$$ en R et D, $\text{200} \text{k}~\$$ en marketing
et $\text{150} \text{k}~\$$ en administration?
<!-- #endregion -->



In [ ]:
X0 = pd.DataFrame(
    {
        "R&D Spend": 50000,
        "Administration": 150000,
        "Marketing Spend": 20000,
        "Florida": 0,
        "New York": 0,
    },
    index=[0],
)

# Normalisation des données
X0_n = norm.transform(X0)

# Prédiction du profit
y0 = reg.predict(X0_n)

print("\t Profits = %0.1f k$" % (y0 / 1000))
